<a href="https://colab.research.google.com/github/rogon666/UMSA/blob/main/AIMLDL_0405.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================================================
# Maestría en Ciencia y Análisis de Datos
# Universidad Mayor de San Andrés
# ----------------------------------------------------------
#           Machine Learning y Deep Learning
# ----------------------------------------------------------
#        Rolando Gonzales Martinez, Agosto 2024
# ==========================================================
#    Estimación y comparación de múltiples modelos de
#  machine learning, incluyend tuneo de hiperparametros
#     y multiples tipo de ensamblaje de modelos ML
# ==========================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, BayesianRidge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor, StackingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA

In [ ]:
# Cargar el conjunto de datos:
url = 'https://raw.githubusercontent.com/rogon666/UMSA/main/AIMLDL/Datos/salarios.csv'

# Cargar los datos en un DataFrame
datos = pd.read_csv(url)

# Mostrar las primeras filas del DataFrame
print(datos.head())

# Preparacion de datos
y = np.log(datos.iloc[:, 0])  # Primera columna como objetivo, aplicando logaritmo
X = datos.iloc[:, 1:]  # Resto de las columnas como características

# Preprocesamiento de datos: Estandarizacion:
preprocesador = Pipeline(steps=[
    ('escalador', StandardScaler())
])

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
proporcion_muestra_test = 0.3 # 30%
semilla = 900
X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(X, y,
                                                                        test_size=proporcion_muestra_test,
                                                                        random_state=semilla)

In [ ]:
# Inicializar diccionario de resultados
resultados = {}

# Modelos machine learning:
modelos = {
    'Regresión Lineal': Pipeline(steps=[('preprocesador', preprocesador),
                                        ('regresor', LinearRegression())]),
    'Bosque Aleatorio': Pipeline(steps=[('preprocesador', preprocesador),
                                        ('regresor', RandomForestRegressor())]),
    'Máquinas soporte vectorial': Pipeline(steps=[('preprocesador', preprocesador),
                           ('regresor', SVR())]),
    'Ridge': Pipeline(steps=[('preprocesador', preprocesador),
                             ('regresor', Ridge())]),
    'Lasso': Pipeline(steps=[('preprocesador', preprocesador),
                             ('regresor', Lasso())]),
    'Bayesian Ridge': Pipeline(steps=[('preprocesador', preprocesador),
                                      ('regresor', BayesianRidge())]),
    'Elastic Net': Pipeline(steps=[('preprocesador', preprocesador),
                                   ('regresor', ElasticNet())]),
    'XGBoost': Pipeline(steps=[('preprocesador', preprocesador),
                               ('regresor', XGBRegressor())]),
    'Bagging': Pipeline(steps=[('preprocesador', preprocesador),
                               ('regresor', BaggingRegressor(estimator=DecisionTreeRegressor(),
                                                             n_estimators=10, random_state=semilla))]),
    'Boosting': Pipeline(steps=[('preprocesador', preprocesador),
                                ('regresor', GradientBoostingRegressor())])
}

# Ensamblaje: Stacking
estimadores = [
    ('Ridge', Ridge()),
    ('Lasso', Lasso()),
    ('Random Forest', RandomForestRegressor()),
    ('SVR', SVR()),
    ('XGBoost', XGBRegressor())
]

modelos['Stacking'] = Pipeline(steps=[('preprocesador', preprocesador),
                                      ('regresor', StackingRegressor(estimators=estimadores,
                                                                     final_estimator=LinearRegression()))])

for nombre, modelo in modelos.items():
    modelo.fit(X_entrenamiento, y_entrenamiento)
    y_pred = modelo.predict(X_prueba)

    rmse = mean_squared_error(y_prueba, y_pred, squared=False)
    mae = mean_absolute_error(y_prueba, y_pred)
    r2 = r2_score(y_prueba, y_pred)

    resultados[nombre] = {'RMSE': rmse, 'MAE': mae, 'R^2': r2}

# Ensamblaje: Cascading
# Modelo 1 (Capa 1): Entrenamiento de RandomForest
modelo_capa1 = RandomForestRegressor(random_state=semilla)
modelo_capa1.fit(X_entrenamiento, y_entrenamiento)

# Usar las predicciones del primer modelo como nuevas características
X_entrenamiento_capa2 = np.column_stack((X_entrenamiento, modelo_capa1.predict(X_entrenamiento)))
X_prueba_capa2 = np.column_stack((X_prueba, modelo_capa1.predict(X_prueba)))

# Modelo 2 (Capa 2): Entrenamiento de LinearRegression sobre las nuevas características
modelo_cascading = Pipeline(steps=[('preprocesador', preprocesador),
                                   ('regresor', LinearRegression())])

modelo_cascading.fit(X_entrenamiento_capa2, y_entrenamiento)

# Predecir con el modelo en cascada
y_pred_cascading = modelo_cascading.predict(X_prueba_capa2)

# Evaluar el modelo en cascada
rmse_cascading = mean_squared_error(y_prueba, y_pred_cascading, squared=False)
mae_cascading = mean_absolute_error(y_prueba, y_pred_cascading)
r2_cascading = r2_score(y_prueba, y_pred_cascading)

resultados['Cascading'] = {'RMSE': rmse_cascading, 'MAE': mae_cascading, 'R^2': r2_cascading}

# Seleccionar el mejor modelo basado en RMSE
mejor_modelo_nombre = min(resultados, key=lambda k: resultados[k]['RMSE'])
mejor_modelo_rendimiento = resultados[mejor_modelo_nombre]

# Mostrar resultados
resultados_df = pd.DataFrame(resultados).T
print("Comparación de Modelos:\n", resultados_df)
print("\nMejor Modelo:", mejor_modelo_nombre)
print("Rendimiento:", mejor_modelo_rendimiento)

In [ ]:
# Modelo 1 (Capa 1): RandomForest con ajuste de hiperparámetros
parametros_rf = {
    'n_estimators': [100, 150, 200],
    'min_samples_split': [2, 3, 5]
}
grid_rf = GridSearchCV(RandomForestRegressor(random_state=semilla),
                       param_grid=parametros_rf, verbose=3, cv=3)
grid_rf.fit(X_entrenamiento, y_entrenamiento)

# Aplicar PCA para reducir la dimensionalidad en la segunda capa
pca = PCA(n_components=2)
X_entrenamiento_capa2 = pca.fit_transform(X_entrenamiento_capa2)
X_prueba_capa2 = pca.transform(X_prueba_capa2)

# Usar las predicciones del mejor modelo de RandomForest como nuevas características
X_entrenamiento_capa2 = np.column_stack((X_entrenamiento, grid_rf.predict(X_entrenamiento)))
X_prueba_capa2 = np.column_stack((X_prueba, grid_rf.predict(X_prueba)))

# Usar las predicciones del primer modelo como nuevas características
X_entrenamiento_capa2 = np.column_stack((X_entrenamiento, modelo_capa1.predict(X_entrenamiento)))
X_prueba_capa2 = np.column_stack((X_prueba, modelo_capa1.predict(X_prueba)))

# Modelo 2 (Capa 2): Entrenado con las nuevas características
modelo_cascading = Pipeline(steps=[('preprocesador', preprocesador),
                                   ('regresor', Ridge())])

modelo_cascading.fit(X_entrenamiento_capa2, y_entrenamiento)

# Predecir con el modelo en cascada mejorado
y_pred_cascading = modelo_cascading.predict(X_prueba_capa2)

# Evaluar el modelo en cascada
rmse_cascading = mean_squared_error(y_prueba, y_pred_cascading, squared=False)
mae_cascading = mean_absolute_error(y_prueba, y_pred_cascading)
r2_cascading = r2_score(y_prueba, y_pred_cascading)

resultados['Cascading Mejorado'] = {'RMSE': rmse_cascading, 'MAE': mae_cascading, 'R^2': r2_cascading}

# Seleccionar el mejor modelo basado en RMSE
mejor_modelo_nombre = min(resultados, key=lambda k: resultados[k]['RMSE'])
mejor_modelo_rendimiento = resultados[mejor_modelo_nombre]

# Mostrar resultados
resultados_df = pd.DataFrame(resultados).T
print("Comparación de Modelos:\n", resultados_df)
print("\nMejor Modelo:", mejor_modelo_nombre)
print("Rendimiento:", mejor_modelo_rendimiento)